## Building a nervous network from scratch


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard # type: ignore
import tensorflow as tf
from tensorflow import keras

In [9]:
df =pd.read_csv('insurance_data.csv')
df.head()



,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [10]:
from sklearn.model_selection import train_test_split
X = df.drop('bought_insurance', axis=1)
y = df['bought_insurance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
x_train_scaled= X_train.copy()
x_train_scaled['age'] = x_train_scaled['age'] / 100
x_test_scaled = X_test.copy() 
x_test_scaled['age'] = x_test_scaled['age'] / 100

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [ ]:
coef = [1.4589982,0.5725685]  
intercept = -0.5305372   

def prediction_function(age, affordibility):
    wigthed_sum = coef[0] * age + coef[1] * affordibility + intercept
    return sigmoid(wigthed_sum)  # Assuming sigmoid is the activation function
    

In [28]:
prediction_function(0.100,0)

np.float64(0.40500890609112866)

In [30]:
def log_loss(y_true, y_pred):
  epsilon = 1e-15
  y_prodicted_new= [max(i,epsilon)for i in y_pred]
  y_prodicted_new= [min(i,epsilon)for i in y_prodicted_new]
  y_prodicted_new= np.array(y_prodicted_new)
  return -np.mean(y_true * np.log(y_prodicted_new) + (1 - y_true) * np.log(1 - y_prodicted_new))

In [74]:
class MyNN:
  def __init__(self):
    self.age = None
    self.X = None
    self.y = None
    self.epochs = None
    self.loss_thesold = None
    self.w1 = None
    self.w2 = None
    self.b = None   


  def fit(self, age,X, y, epochs,loss_thesold):
    self.age = age
    self.X = X
    self.y = y
    self.epochs = epochs
    self.loss_thesold = loss_thesold
    self.w1, self.w2, self.b = self.gradient_descent(age, X, y, epochs)
    return self.w1, self.w2, self.b
  

  def predict(self, x_test):
    weighted_sum = self.w1 * x_test['age'] + self.w2 * x_test['affordibility'] + self.b
    y_pred = sigmoid(weighted_sum)
    return y_pred
  
  def gradient_descent(self, age, affordibility, y_true, epochs):
    w1=w2=1
    b=0
    rate=0.5
    n=len(age)
    for i in range(epochs):
      weighted_sum=w1*age+w2*affordibility+b
      y_pred=sigmoid(weighted_sum)
      loss=log_loss(y_true, y_pred)
      w1d = (1/n)*np.dot(age, (y_pred - y_true))
      w2d = (1/n)*np.dot(affordibility, (y_pred - y_true))
      bd = (1/n)*np.mean(y_pred-y_true)
      w1=w1-rate*w1d
      w2=w2-rate*w2d
      b=b-rate*bd
      print(f"Epoch {i+1}: w1={w1}, w2={w2}, b={b}, loss={loss}")
    return w1, w2, b

In [75]:
my_nn = MyNN()
my_nn.fit(x_train_scaled['age'], x_train_scaled['affordibility'], y_train, epochs=1000, loss_thesold=0.01)

Epoch 1: w1=0.9859032377975142, w2=0.9358875321595159, b=-0.005474255660516211, loss=18.17830336574246
Epoch 2: w1=0.9736205108213136, w2=0.8759220821903565, b=-0.010709514626322273, loss=18.17830336574246
Epoch 3: w1=0.9630947738107005, w2=0.8199884105280303, b=-0.015713537748593853, loss=18.17830336574246
Epoch 4: w1=0.9542581400440379, w2=0.7679447732266789, b=-0.020495437149540482, loss=18.17830336574246
Epoch 5: w1=0.9470340792531713, w2=0.7196281438850044, b=-0.025065389272660187, loss=18.17830336574246
Epoch 6: w1=0.9413396224450017, w2=0.6748594402678895, b=-0.0294343499404209, loss=18.17830336574246
Epoch 7: w1=0.9370874649470344, w2=0.63344850048527, b=-0.03361378487132254, loss=18.17830336574246
Epoch 8: w1=0.9341878880799603, w2=0.5951986237013125, b=-0.03761542530364202, loss=18.17830336574246
Epoch 9: w1=0.9325504488450893, w2=0.5599105592973519, b=-0.04145105465941056, loss=18.17830336574246
Epoch 10: w1=0.9320854127543118, w2=0.5273858890168599, b=-0.04513232893757052, 

(np.float64(5.731032890110925),
 np.float64(0.12099220244762979),
 np.float64(-1.9272382478842747))

In [76]:
import pickle

# Save the model to a file
with open('model.pkl', 'wb') as file:
  pickle.dump(my_nn, file)